# Agent Onboarding buddy

<img src="media/autogen-onboarding.png" alt="Image description" width="700">

This notebook uses the following agents:

1. __User proxy__: Represents the human user who provides an initial prompt to the agent - in our case, a new hire ⭐
2. __Onboarding buddy__: An agent onboarding the user, by answering questions based on the company's internal sources and guiding through mandatory assignments for better understanding ⭐
3. __Memory manager__: An agent responsible for memory management of onboarding sessions, and memories/personal preferences of the user in the database. Thanks to the memory manager, the conversation can restart exactly where it was left off ⭐

## Setup
Install pyautogen with llm option:
```bash
pip install "pyautogen[lmm]>=0.2.3"
```

In [26]:
from dotenv import load_dotenv, find_dotenv
from azure.identity import DefaultAzureCredential
from azure.cosmos import CosmosClient, exceptions
import autogen
from autogen.cache import Cache
import os

load_dotenv(find_dotenv())

config_list = autogen.config_list_from_json(
    "../OAI_CONFIG_LIST.json",
    filter_dict={
        "model": ["gpt-4o"],
    },
)

settings = {
    'host': os.getenv('ACCOUNT_HOST'),
    'database_id': os.getenv('COSMOS_DATABASE'),
    'container_id': os.getenv('COSMOS_CONTAINER_EMPLOYEE'),
    'tenant_id': os.getenv('TENANT_ID'),
    'client_id': os.getenv('CLIENT_ID'),
    'client_secret': os.getenv('CLIENT_SECRET')
}


for key, value in settings.items():
    if value is None:
        raise ValueError(f"Missing environment variable for {key}")

In [9]:
onboarding_buddy_prompt = """
As a friendly and approachable onboarding buddy at Contoso, your task is to create a warm, personalized, and engaging onboarding experience for a new hire. Start by greeting them enthusiastically, as if welcoming a friend, and introduce yourself casually.
Guide the new hire through each onboarding topic as if you're having a chat over coffee, breaking down complex information into bite-sized, relatable pieces. When explaining topics like code of conduct, company culture, and policies, use real-life examples and anecdotes to make the information more engaging and memorable.
Encourage questions and open dialogue, making the new hire feel comfortable to share their thoughts and concerns. Regularly check in on their understanding in a casual manner before moving to the next topic.
Your goal is to make the new hire feel like they're talking to a knowledgeable friend who's genuinely excited to help them settle into their new role at Contoso.
Use the following guidelines:
- Ask for their Employee ID to retrieve their information from the database.
- Use the "search" function to access relevant information from the knowledge base about Contoso-specific processes and policies.
For topics like code of conduct, company culture, policies etc, provide detailed explanations and examples. Do not send all the official information at once, but rather provide it in manageable portions.
- Regularly check for understanding and encourage questions.
- Guide the new hire through each stage of the onboarding process, ensuring all necessary steps are completed.
- Do not move on to the next topic until the new hire has confirmed understanding.
- Log in all the progress that employee has made/all the topics that the employee has learned as well as any relevant employee information in the database by working with the memory manager.
- At the start of new conversation, pay attention to the memory manager's response to know what the new hire has learned so far.
- DO NOT MAKE ANYTHING UP. If you are unsure about any information, use the "search" function to find the correct information.
"""


memory_manager_prompt = """
When asked to write to memory:
a. First, read the existing memory.
b. Evaluate if the new information is not already represented in the existing memories and is important enough to log it in.
c. If it's new and important, rewrite the existing memories to incorporate the information.
Memory format:

Memories are formatted as a series of statements separated by the '|' character.
Example: "User is French| User likes dogs| User has 2 kids"
Updating memory:

Incorporate new information by modifying existing statements or adding new ones.
Remove outdated or contradictory information when necessary.
Keep memories short, succinct, and accurate.

DO NOT engage in any form of conversation beyond this task. DO NOT TALK TO THE USER.
"""

## Defining Agents

The user proxy agent is the new hire who would interact within the conversation. We are also interested in creating an agent who is capable of executing code. For this, we create an "executor" agent who will use a Docker container for code execution. Additionally, we create the main Onboarding Buddy agent and Memory Manager agent. Memory manager is solely responsible for interacting with memories, but will not participate in the conversations.

We will use the "Group Chat" structure from autogen such that all the agents are part of the same conversation. The Chat Manager is responsible for routing the message to the correct agent. For the manager to have a better understanding of the role of every agent, we include the introductions.

In [10]:
import tempfile
from autogen import ConversableAgent
from autogen.coding import DockerCommandLineCodeExecutor

llm_config = {"config_list": config_list, "cache_seed": 42}
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="The new hire.",
    human_input_mode="ALWAYS",
    code_execution_config=False
)

temp_dir = tempfile.TemporaryDirectory()

executor = DockerCommandLineCodeExecutor(
    image="python:3.12-slim",
    timeout=10,
    work_dir=temp_dir.name
)

onboarding_buddy = autogen.ConversableAgent(
    name="onboarding_buddy",
    system_message= onboarding_buddy_prompt,
    llm_config=llm_config
)

memory_manager = autogen.ConversableAgent(
    name="memory_manager",
    system_message=memory_manager_prompt,
    llm_config=llm_config
)

code_executor_agent_using_docker = autogen.AssistantAgent(
    "code_executor_agent_docker",
    llm_config=False,
    code_execution_config={"executor": executor},
    human_input_mode="NEVER",
)

groupchat = autogen.GroupChat(agents=[user_proxy, onboarding_buddy, code_executor_agent_using_docker, memory_manager], messages=[], max_round=50, send_introductions=True)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)


## Defining functions 

- Retrieve employee information from the database
- Read, write and update memory 
- Do RAG over company-specific documents for onboarding

In [24]:
import json
import requests
import azure.cosmos.documents as documents
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey
import json

HOST = settings['host']
DATABASE_ID = settings['database_id']
CONTAINER_ID = settings['container_id']
TENANT_ID = settings['tenant_id']
CLIENT_ID = settings['client_id']
CLIENT_SECRET = settings['client_secret']

credential = DefaultAzureCredential()
client = cosmos_client.CosmosClient(HOST, credential=credential)
db = client.get_database_client(DATABASE_ID)
container = db.get_container_client(CONTAINER_ID)

@code_executor_agent_using_docker.register_for_execution()
@onboarding_buddy.register_for_llm(name="get_employee_info", description="Retrieve employee data from the database, including the start date and mandatory exam asignments.")
def get_employee_info(EmployeeId: str) -> str:
    try:
        query = f"SELECT * FROM c WHERE c.EmployeeId = '{EmployeeId}'"
        items = list(container.query_items(query=query, enable_cross_partition_query=True))

        if items:
            employee_info = items[0]
            return json.dumps(employee_info, default=str)
        else:
            error_message = {'error': f"Employee with ID '{EmployeeId}' not found."}
            return json.dumps(error_message)

    except exceptions.CosmosHttpResponseError as e:
        error_message = {'error': str(e)}
        return json.dumps(error_message)

CosmosHttpResponseError: (Forbidden) Request blocked by Auth cosmosdb999 : Request is blocked because principal [e746e927-ef87-47ff-9dd6-d693297e808a] does not have required RBAC permissions to perform action [Microsoft.DocumentDB/databaseAccounts/readMetadata] on resource [/]. Learn more: https://aka.ms/cosmos-native-rbac.
ActivityId: 4a4e87fd-9510-409c-986e-29612aa695fb, Microsoft.Azure.Documents.Common/2.14.0
Code: Forbidden
Message: Request blocked by Auth cosmosdb999 : Request is blocked because principal [e746e927-ef87-47ff-9dd6-d693297e808a] does not have required RBAC permissions to perform action [Microsoft.DocumentDB/databaseAccounts/readMetadata] on resource [/]. Learn more: https://aka.ms/cosmos-native-rbac.
ActivityId: 4a4e87fd-9510-409c-986e-29612aa695fb, Microsoft.Azure.Documents.Common/2.14.0

In [4]:
import json
from azure.cosmos import exceptions
from typing import Dict, Any, Optional

@code_executor_agent_using_docker.register_for_execution()
@memory_manager.register_for_llm(name="read_memory", description="Read the conversation history of an employee.")
def read_memory(employee_id: str) -> Dict[str, Any]:
    try:
        query = "SELECT c.ConversationHistory FROM c WHERE c.EmployeeId = @employee_id"
        parameters = [{"name": "@employee_id", "value": employee_id}]
        items = list(container.query_items(
            query=query,
            parameters=parameters,
            enable_cross_partition_query=True
        ))

        if items:
            conversation_history = items[0].get("ConversationHistory", "")
            return {"conversation_history": conversation_history}
        else:
            return {"error": f"Employee with ID '{employee_id}' not found."}

    except exceptions.CosmosHttpResponseError as e:
        return {"error": str(e)}

@code_executor_agent_using_docker.register_for_execution()
@memory_manager.register_for_llm(name="write_memory", description="Write an additional message to the conversation history of an employee.")
def write_memory(employee_id: str, new_memory: str) -> Dict[str, Any]:
    try:
        query = "SELECT * FROM c WHERE c.EmployeeId = @employee_id"
        parameters = [{"name": "@employee_id", "value": employee_id}]
        items = list(container.query_items(
            query=query,
            parameters=parameters,
            enable_cross_partition_query=True
        ))

        if items:
            item = items[0]
            current_history = item.get("ConversationHistory", "")
            updated_history = f"{current_history}|{new_memory}" if current_history else new_memory
            item["ConversationHistory"] = updated_history
            updated_item = container.upsert_item(body=item)
            return {"updated_item": updated_item}
        else:
            return {"error": f"Employee with ID '{employee_id}' not found."}

    except exceptions.CosmosHttpResponseError as e:
        return {"error": str(e)}

@code_executor_agent_using_docker.register_for_execution()
@memory_manager.register_for_llm(name="update_memory", description="Update the conversation history of an employee with the new history/override.")
def update_memory(employee_id: str, new_history: str) -> Dict[str, Any]:
    try:
        query = "SELECT * FROM c WHERE c.EmployeeId = @employee_id"
        parameters = [{"name": "@employee_id", "value": employee_id}]
        items = list(container.query_items(
            query=query,
            parameters=parameters,
            enable_cross_partition_query=True
        ))

        if items:
            item = items[0]
            item["ConversationHistory"] = new_history
            updated_item = container.upsert_item(body=item)
            return {"updated_item": updated_item}
        else:
            return {"error": f"Employee with ID '{employee_id}' not found."}

    except exceptions.CosmosHttpResponseError as e:
        return {"error": str(e)}

NameError: name 'code_executor_agent_using_docker' is not defined

In [5]:
from openai import AzureOpenAI
from typing import List, Dict
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import (
    VectorizedQuery,
    QueryType,
    QueryCaptionType,
    QueryAnswerType,
)
import os

def get_embedding(query: str) -> List[float]:
    client = AzureOpenAI(
        api_key=os.getenv("AZURE_OAI_KEY"),
        azure_endpoint=os.getenv("AZURE_OAI_ENDPOINT"),
        api_version="2024-02-01"
    )
    try:
        response = client.embeddings.create(
            input=query,
            model="embedding",
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"Error getting embedding: {e}")
        return []

def retrieve_documentation(query: str) -> List[Dict]:
    try:
        embedding = get_embedding(query)
        if not embedding:
            return []

        search_client = SearchClient(
            endpoint=os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"],
            index_name="onboarding",
            credential=AzureKeyCredential(os.getenv("AZURE_SEARCH_KEY"))
        )

        vector_query = VectorizedQuery(
            vector=embedding, k_nearest_neighbors=3, fields="contentVector"
        )

        results = search_client.search(
            search_text=query,
            vector_queries=[vector_query],
            query_type=QueryType.SEMANTIC,
            semantic_configuration_name="azureml-default",
            query_caption=QueryCaptionType.EXTRACTIVE,
            query_answer=QueryAnswerType.EXTRACTIVE,
            top=3,
        )

        docs = [
            {
                "id": doc["id"],
                "title": doc["title"],
                "content": doc["content"],
                "url": doc["url"],
            }
            for doc in results
        ]

        return docs
    except Exception as e:
        print(f"Error retrieving documentation: {e}")
        return []

@code_executor_agent_using_docker.register_for_execution()
@onboarding_buddy.register_for_llm(name="search", description="Search the knowledge base for documentation to answer the question of the user.")
def search(query: str) -> List[Dict]:
    return retrieve_documentation(query)

NameError: name 'code_executor_agent_using_docker' is not defined

In [15]:
chat_result = user_proxy.initiate_chat(manager,message="Hi, I am a new hire")

User_proxy (to chat_manager):

Hi, I am a new hire

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
onboarding_buddy (to chat_manager):

Hey there! Welcome to Contoso! 😊 I'm so glad you're here and excited to help you get started. I'm your onboarding buddy, and I'm here to make your onboarding experience as smooth and enjoyable as possible. 

To kick things off, could you please share your Employee ID with me? I'll use it to retrieve your information from our database and get things rolling. 

Looking forward to helping you settle in!

--------------------------------------------------------------------------------
User_proxy (to chat_manager):

123

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
memory_manager (to chat_manager):

***** Suggested tool call (call_sa7hyFTT3nJQbquSFprwGLma): read_memory *****
Arguments: 
{"employee_id":"123"}
*********************